In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.pipeline import Pipeline
from scikeras.wrappers import KerasClassifier
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
import pickle

In [2]:
data=pd.read_csv('Churn_Modelling.csv')
data = data.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)

In [3]:
label_encoder_gender = LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])

onehot_encoder_geo = OneHotEncoder(handle_unknown='ignore')
geo_encoded = onehot_encoder_geo.fit_transform(data[['Geography']]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=onehot_encoder_geo.get_feature_names_out(['Geography']))

In [4]:
data = pd.concat([data.drop('Geography', axis=1), geo_encoded_df], axis=1)


In [5]:

X = data.drop('Exited', axis=1)
y = data['Exited']

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [7]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [8]:
#save models
# Save encoders and scaler for later use
with open('label_encoder_gender.pkl', 'wb') as file:
    pickle.dump(label_encoder_gender, file)

with open('onehot_encoder_geo.pkl', 'wb') as file:
    pickle.dump(onehot_encoder_geo, file)

with open('scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)

In [9]:
## Define a function to create the model and try different parameters(KerasClassifier)

def create_model(neurons=32,layers=1):
    model=Sequential()
    model.add(Dense(neurons,activation='relu',input_shape=(X_train.shape[1],)))

    for _ in range(layers-1):
        model.add(Dense(neurons,activation='relu'))

    model.add(Dense(1,activation='sigmoid'))
    model.compile(optimizer='adam',loss="binary_crossentropy",metrics=['accuracy'])

    return model



In [10]:
## Create a Keras classifier
model=KerasClassifier(layers=1,neurons=32,build_fn=create_model,verbose=1)

In [11]:
# Define the grid search parameters
param_grid = {
    'neurons': [16, 32, 64, 128],
    'layers': [1, 2],
    'epochs': [50, 100]
}

In [12]:
# Perform grid search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3,verbose=1)
grid_result = grid.fit(X_train, y_train)

# Print the best parameters
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Fitting 3 folds for each of 16 candidates, totalling 48 fits


c:\Users\Uyama\Downloads\JANE-UDEMY\NLP\annclassNLP\venv\Lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)




Epoch 1/100


250/250 [==============================] - 2s 2ms/step - loss: 0.5152 - accuracy: 0.7780
Epoch 2/100
250/250 [==============================] - 1s 2ms/step - loss: 0.4429 - accuracy: 0.8023
Epoch 3/100
250/250 [==============================] - 1s 2ms/step - loss: 0.4261 - accuracy: 0.8110
Epoch 4/100
250/250 [==============================] - 1s 2ms/step - loss: 0.4144 - accuracy: 0.8195
Epoch 5/100
250/250 [==============================] - 1s 2ms/step - loss: 0.4032 - accuracy: 0.8255
Epoch 6/100
250/250 [==============================] - 1s 2ms/step - loss: 0.3927 - accuracy: 0.8321
Epoch 7/100
250/250 [==============================] - 1s 2ms/step - loss: 0.3837 - accuracy: 0.8384
Epoch 8/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3760 - accuracy: 0.8413
Epoch 9/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3697 - accuracy: 0.8471
Epoch 10/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3646 - ac